# Bond Breaking
Here we use the VQE to break the H2 bond, as defined by the `distort_exp_input.yml` file.

Before we begin, let's view the input yml file

In [ ]:
!cat experiment_inputs/distort_exp_input.yml

Note that this is very similar to the previous yml file except we nopw have included the `distortion_dict`. This dictionary specifies how the bond-strectching is performed. 

`equilibrium_geom` is the initial geometry of the molecule

`atom_bond_indices` specify which two atoms makeup the bond which is to be strecthed.

`stretch_params` is a list containing three terms. The first is the distance the two atoms are to be brought closer, the second is the distance the two atoms are to be brought apart and the final is the number of distortions.

When running the bond-breaking from the `distortion_dict`, initially all atoms in the molecule are divided into two clusters, depending on which of the two atoms (which make up the bond) they're closest to. Then when streching/compressing the bond, all atoms on the same cluster are moved together.

In [ ]:
import qpfas
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Run VQE

In [ ]:
client = qpfas.workflow.DaskDAG.create_client(dask_scheduler_uri="tcp://dask-scheduler:8786")

In [ ]:
YAML_FILE = os.path.join(os.getcwd(), 'experiment_inputs','distort_exp_input.yml')

graph, keys_to_gather, exp_params, dag_nodes = qpfas.workflow.EfficientDAG.load_from_yaml(YAML_FILE)

print("Number of Experiments:", len(exp_params))

print("\nDAG Nodes:")
for i in dag_nodes:
    print(f"{i}: {dag_nodes[i]}")
    
print("\nKeys Gathered:")
print(keys_to_gather)

In [ ]:
# to view the DASK graph
# qpfas.workflow.DaskDAG.visualize_dag(graph)  

In [ ]:
results = qpfas.workflow.DaskDAG.get(client, graph, keys_to_gather)
qpfas.workflow.EfficientDAG.write_to_file("H2_dist_exp_output", exp_params, results)

In [ ]:
!head -40 H2_dist_exp_output.json

Note that the running the bond-breaking workflow produces an xyz file with the distortions in them.

In [ ]:
!cat H2_distortions.xyz

## Analysis of Data

In [ ]:
exp_df = pd.read_json("H2_dist_exp_output.json")
exp_df

In [ ]:
print(exp_df.columns)
print(exp_df.iloc[0])

In [ ]:
%matplotlib notebook

vqe_energy = [i["converged_energy"] for i in exp_df["vqe_output"]]

plt.figure(figsize=(6, 4))

plt.plot(exp_df["bond_distance"], vqe_energy, label="VQE", marker='o')
plt.plot(exp_df["bond_distance"], exp_df["hf_energy"], label="HF", marker='o')
plt.plot(exp_df["bond_distance"], exp_df["fci_energy"], label="FCI", ls='dashed', c='k')

plt.legend()
plt.xlabel("Bond Distance (A)")
plt.ylabel("Energy (H)")
plt.show()

In [ ]:
%matplotlib notebook

plt.figure(figsize=(6, 4))

plt.plot(exp_df["bond_distance"], vqe_energy-exp_df["fci_energy"], label="VQE", marker='o')
plt.plot(exp_df["bond_distance"], exp_df["hf_energy"]-exp_df["fci_energy"], label="HF", marker='o')

plt.legend()
plt.xlabel("Bond Distance (A)")
plt.ylabel("Error (H)")
plt.yscale("log")
plt.show()